In [1]:
import os
import json
from openai import OpenAI
client = OpenAI()

client.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def get_current_weather(location, unit="섭씨"):
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info, ensure_ascii=False)


In [3]:
assistant = client.beta.assistants.create(
    name="날씨봇",
    instructions="당신은 날씨 챗봇입니다. 사용자의 질문에 대답하기 위해 주어진 function들을 사용하세요.",
    tools = [
        {
            "type": "function",
            "function":
            {
                "name": "get_current_weather",
                "description": "특정 지역의 날씨를 알려줍니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "지역이름. 예) 서울, 부산, 제주도",
                        },
                        "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
                    },
                    "required": ["location"],
                },
            }
        }
    ],
    model="gpt-4o-mini"
)
assistant

Assistant(id='asst_YTvU9mnm8xTvzdZUfoOyU4uX', created_at=1753247875, description=None, instructions='당신은 날씨 챗봇입니다. 사용자의 질문에 대답하기 위해 주어진 function들을 사용하세요.', metadata={}, model='gpt-4o-mini', name='날씨봇', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_current_weather', description='특정 지역의 날씨를 알려줍니다.', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': '지역이름. 예) 서울, 부산, 제주도'}, 'unit': {'type': 'string', 'enum': ['섭씨', '화씨']}}, 'required': ['location']}, strict=False), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0, reasoning_effort=None)

In [4]:
thread = client.beta.threads.create()
thread

/tmp/ipykernel_63685/1159370957.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


Thread(id='thread_eScDOeC7cOI3Vmy3CudKTsZ4', created_at=1753247880, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="서울의 날씨는 어때?"
)

/tmp/ipykernel_63685/2592962367.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [6]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

/tmp/ipykernel_63685/2546265667.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread_messages = client.beta.threads.messages.list(thread.id)


[Message(id='msg_J9bTrFXNCH2TF9xeqtBM1o21', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='서울의 날씨는 어때?'), type='text')], created_at=1753247884, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_eScDOeC7cOI3Vmy3CudKTsZ4')]


In [7]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

/tmp/ipykernel_63685/3463038992.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [8]:
run_id = run.id

In [9]:
import time

while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run_id
    )
    if run.required_action is not None:
        break

/tmp/ipykernel_63685/3388113875.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [10]:
print(run.required_action.submit_tool_outputs.tool_calls[0])

RequiredActionFunctionToolCall(id='call_17gXiEcotkEOuBvg0WjIJSVo', function=Function(arguments='{"location":"서울","unit":"섭씨"}', name='get_current_weather'), type='function')


In [11]:
import json

available_functions = {
    "get_current_weather": get_current_weather,
}

for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    tool_call_id = tool_call.id
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args['location']
    )

print(function_response)

{"location": "서울", "temperature": "24", "unit": "섭씨", "forecast": ["sunny", "windy"]}


In [12]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
      {
        "tool_call_id": tool_call_id,
        "output": json.dumps(function_response, ensure_ascii=False)
      }
    ]
)
run_id = run.id

/tmp/ipykernel_63685/3241975149.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.submit_tool_outputs(


In [13]:
import time

while run.status != "completed"  :
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    time.sleep(0.1)

/tmp/ipykernel_63685/1699223118.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [14]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

/tmp/ipykernel_63685/1031825989.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(


In [15]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data[0].content[0].text.value)

/tmp/ipykernel_63685/4045452831.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread_messages = client.beta.threads.messages.list(thread.id)


서울의 현재 날씨는 맑고 바람이 불고 있으며, 기온은 24도입니다.
